# Links to Dataset

###### Kaggle Dataset - https://www.kaggle.com/datasets/sanjanchaudhari/employees-performance-for-hr-analytics

# Importing Modules

In [1]:
import pypyodbc as odbc
import pandas as pd

# Linking to SQL Database and Converting to Pandas Dataframe

## Linking to SQL Database

In [2]:
DRIVER_NAME = "SQL Server"
SERVER_NAME = "LAPTOP-CT9C24F3"
DATABASE_NAME = "HR Analytics"

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Trust_Connection=yes;
"""

In [3]:
conn = odbc.connect(connection_string)
cursor = conn.cursor()

### Making an empty Dataframe with the SQL Dataset Headers

In [4]:
res = cursor.execute("SELECT * FROM sys.columns WHERE object_id = OBJECT_ID('Employees')")

headers = []
for i in res:
    headers.append(i[1])

df = pd.DataFrame(columns=headers)
df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met_more_than_80,awards_won,avg_training_score


### Populating the Dataframe with SQL Dataset

In [6]:
res = cursor.execute("SELECT * FROM Employees")

column_lists = []
for header in headers:
    column_lists.append([])

for i in res:
    n = 0
    for j in i:
        new_list = list(column_lists[n])
        new_list.append(j)
        column_lists[n] = new_list
        n += 1

In [9]:
n = 0
for column in column_lists:
    df[df[df.columns[n]].name] = column
    n += 1

df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met_more_than_80,awards_won,avg_training_score
0,78283.0,Operations,region_12,Bachelors,f,sourcing,1.0,28.0,3.0,4.0,0.0,0.0,57.0
1,7656.0,Sales & Marketing,region_10,Masters & above,m,other,1.0,46.0,4.0,4.0,1.0,0.0,59.0
2,5143.0,Operations,region_29,Bachelors,m,sourcing,1.0,35.0,4.0,6.0,1.0,0.0,64.0
3,2981.0,Operations,region_9,Bachelors,f,sourcing,1.0,36.0,5.0,8.0,1.0,0.0,59.0
4,40452.0,Sales & Marketing,region_22,Masters & above,m,sourcing,1.0,53.0,3.0,17.0,0.0,0.0,47.0
